In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
# @Time : ${2023-6-27} ${21:05}
# @Author : Mathew Jin
# @File : ${run_model.py}
# chmod +x run_model.py
# python3 scripts/run_model3.py --process=0 --rawname=1 --resume=0 --anchoring=0
'''
1.TRA
https://arxiv.org/pdf/2106.12950.pdf
https://github.com/microsoft/qlib/blob/main/examples/benchmarks/TRA/src/model.py
1.1 HIST
https://arxiv.org/pdf/2110.13716.pdf
https://github.com/Wentao-Xu/HIST
2.Lightgbm
https://github.com/microsoft/LightGBM/blob/master/examples/python-guide/plot_example.py
https://lightgbm.readthedocs.io/en/latest/pythonapi/lightgbm.plot_tree.html
3.other factors
'''
import argparse
import torch
import torch.nn as nn
import numpy as np
import itertools , random , os, shutil , gc , time , h5py , yaml
from tqdm import tqdm
from copy import deepcopy
from torch.optim.swa_utils import AveragedModel , update_bn
from scripts.util.environ import get_logger
from scripts.util.basic import process_timer , FilteredIterator , lr_cosine_scheduler , versatile_storage
from scripts.util.multiloss import multiloss_calculator 
from scripts.data_util.ModelData import ModelData
from scripts.trainer.config import *
from scripts.function.basic import *
from scripts.nn.My import *
# from audtorch.metrics.functional import *

try:
    parser = trainer_parser().parse_args()
except:
    parser = trainer_parser().parse_args(args=[])

logger = get_logger()
config = train_config(parser = parser , do_process=True)
set_trainer_configs(config)
time_recorder = process_timer(True)
storage_model = versatile_storage(config.storage_type)

usage: ipykernel_launcher.py [-h] [--process PROCESS] [--rawname RAWNAME]
                             [--resume RESUME] [--anchoring ANCHORING]
ipykernel_launcher.py: error: unrecognized arguments: --f=c:\Users\jinmeng\AppData\Roaming\jupyter\runtime\kernel-v2-18492CwSBtrcuOHON.json


--What process would you want to run? 0: all, 1: train only (default), 2: test only , 3: copy to instance
--Process Queue : Data + Train + Test + Instance
--Multiple model path of LSTM_day_SHORTTEST exists, input [yes] to resume training, or start a new one!
--Confirm Resume Training!
--Model_name is set to LSTM_day_SHORTTEST!


In [52]:
list_converge

<function scripts.function.basic.list_converge(l, n=None, eps=None)>

In [50]:
config.train_params['criterion']['score']

{'loss': 'ccc',
 'score': 'pearson',
 'penalty': {'hidden_orthogonality': {'lamb': 0.001},
  'tra_ot_penalty': {'lamb': 0.01, 'rho': 0.999}}}

In [15]:
import time
a = time.ctime()
time.time() - time.mktime(time.strptime(a))
b = time.time()
time.ctime()

'Sun Jan 28 10:20:14 2024'

In [47]:
import itertools
class FilteredIterator:
    def __init__(self, iterable, condition):
        self.iterable  = iter(iterable)
        if callable(condition):
            self.condition = condition
        else:
            self.condition = iter(condition)
    def __iter__(self):
        return self
    def __next__(self):
        while True:
            item = next(self.iterable)
            cond = self.condition(item) if callable(self.condition) else next(self.condition)
            if cond: return item
model_iter = list(itertools.product(range(3) , range(2)))
models_trained = np.array([True,False,False,False,False,False])
for i,(model_date,model_num) in enumerate(model_iter):
    print(i,model_date,model_num)
model_iter = FilteredIterator(model_iter, models_trained == 0)
for i,x in enumerate(model_iter):
    print(i,x)

0 0 0
1 0 1
2 1 0
3 1 1
4 2 0
5 2 1
0 (0, 1)
1 (1, 0)
2 (1, 1)
3 (2, 0)
4 (2, 1)


In [40]:
list(itertools.product(range(3) , range(2))) , models_trained == 0

([(0, 0), (0, 1), (1, 0), (1, 1), (2, 0), (2, 1)],
 array([False,  True,  True,  True,  True,  True]))

In [28]:
list(itertools.product(range(3) , range(2)))

[(0, 0), (0, 1), (1, 0), (1, 1), (2, 0), (2, 1)]

In [8]:
getattr(config , 'TRAIN_PARAM' , None)

{'dataloader': {'random_seed': None,
  'random_tv_split': True,
  'sample_method': 'train_shuffle',
  'train_ratio': 0.8},
 'trainer': {'optimizer': {'name': 'Adam', 'param': {}},
  'scheduler': {'name': 'cycle',
   'param': {'base_lr': 1e-07, 'step_size_up': 4}},
  'learn_rate': {'base': 0.005,
   'ratio': {'attempt': [1, 0.1, 10, 0.01, 100],
    'round': [1.0],
    'transfer': 0.1},
   'reset': {'num_reset': 2,
    'trigger': 40,
    'recover_level': 1.0,
    'speedup2x': True}},
  'nanloss': {'retry': 2},
  'gradient': {'clip_value': 10.0},
  'retrain': {'attempts': 4, 'min_epoch': 20, 'min_epoch_round': 10}},
 'criterion': {'loss': 'ccc',
  'score': 'pearson',
  'penalty': {'hidden_orthogonality': {'lamb': 0.001},
   'tra_ot_penalty': {'lamb': 0.01, 'rho': 0.999}}},
 'transfer': False,
 'output_types': ['best', 'swalast', 'swabest'],
 'multitask': {'type': 'hybrid',
  'param_dict': {'dwa': {'tau': 2},
   'ruw': {'phi': None},
   'ewa': {},
   'gls': {},
   'rws': {},
   'hybrid': {

In [47]:
import torch
use_device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
class Device:
    def __init__(self , device = use_device) -> None:
        self.device = device
    def __call__(self, *args: Any, **kwds: Any) -> Any:
        args = self.to_device(args)
        kwds = self.to_device(kwds)
        return args , kwds
    def _to(self , x):
        if isinstance(x , (list,tuple)):
            return type(x)(self._to(v) for v in x)
        elif isinstance(x , (dict)):
            return {k:self._to(v) for k,v in x.items()}
        else:
            return x.to(self.device) if x is not None else None
    def torch_nans(self,*args,**kwargs):
        return torch.ones(*args , device = self.device , **kwargs).fill_(torch.nan)
    def torch_zeros(self,*args , **kwargs):
        return torch.zeros(*args , device = self.device , **kwargs)
    def torch_ones(self,*args,**kwargs):
        return torch.ones(*args , device = self.device , **kwargs)
    def torch_arange(self,*args,**kwargs):
        return torch.arange(*args , device = self.device , **kwargs)
    def print_cuda_memory(self):
        print(f'Allocated {torch.cuda.memory_allocated(self.device) / 1024**3:.1f}G, '+\
              f'Reserved {torch.cuda.memory_reserved(self.device) / 1024**3:.1f}G')
        
d = Device()
d.arange(10)

tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])